In [1]:
from attr_rec import *
from IPython.display import display
import ipywidgets as widgets
import pandas as pd
import re

Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
att_df = pd.read_json('../data/attraction/attractions.json',orient='records')

## User details and Preferences

In [3]:
options_place = list(att_df.groupby('city').size().reset_index().sort_values([0], ascending=True).city.values)

uname = widgets.Text(description="Tên người dùng", style=dict(description_width='111px'))
place = widgets.Dropdown(options=options_place, description='Điểm đến (Tỉnh/TP)', style=dict(description_width='111px'))
budget = widgets.IntRangeSlider(min=0, max=10000000, step=10000, value=[0, 10000000], description="Kinh phí (VND)", style=dict(description_width='111px'))
v1 = widgets.VBox([uname,place,budget])
v1.layout.width = '400px'
start = widgets.DatePicker(description='Ngày bắt đầu', disabled=False, style=dict(description_width='100px'))
end = widgets.DatePicker(description='Ngày kết thúc', disabled=False, style=dict(description_width='100px'))
v2 = widgets.VBox([start,end])
out = widgets.HBox([v1,v2])
display(out)

In [4]:
category_df = att_df.groupby('category').size().reset_index().sort_values([0],ascending=False)[:20]
categories = list(category_df.category.values)

cat_rat = dict()
def on_button_clicked(b):
    if b.description in cat_rat:
        return
    print(b.description)
    slider = widgets.IntSlider(min=0,max=5,step=1,description='Rate')
    display(slider)
    cat_rat[b.description] = slider
    if(len(cat_rat) < 5):
        print("Rate {x} more!\n".format(x=5-len(cat_rat)))
    
but_layout = widgets.Layout(width='100%', height='100px')
but_items = [widgets.Button(description=c, layout=but_layout) for c in categories]
on_clk = [item.on_click(on_button_clicked) for item in but_items]
c1 = widgets.VBox([i for i in but_items[:4]])
c2 = widgets.VBox([i for i in but_items[4:8]])
c3 = widgets.VBox([i for i in but_items[8:12]])
c4 = widgets.VBox([i for i in but_items[12:16]])
c5 = widgets.VBox([i for i in but_items[16:]])
buttons = widgets.HBox([c1,c2,c3,c4,c5])
print("Select and rate atleast 5 categories and rate them:")
display(buttons)

Select and rate atleast 5 categories and rate them:


Đồ uống, thư giãn


IntSlider(value=0, description='Rate', max=5)

Rate 4 more!

Trung tâm thương mại


IntSlider(value=0, description='Rate', max=5)

...

IntSlider(value=0, description='Rate', max=5)

Rate 2 more!

Thủy cung, vui chơi giải trí


IntSlider(value=0, description='Rate', max=5)

Rate 1 more!

Tổ hợp thể thao, đi dạo ngắm cảnh


IntSlider(value=0, description='Rate', max=5)

In [5]:
user_name = re.sub(' ','_',uname.value.lower())
city = place.value
(low,high) = tuple([float(i) for i in budget.value])
begin_date = start.value
end_date = end.value
cat_rating = dict()
for key, value in cat_rat.items():
    cat_rating[key] = float(value.value)

## Model recommendation and filtering

In [6]:
filename, user, rbm_att = get_recc(att_df, cat_rating)
filtered = filter_df(filename, user, low, high, city, att_df)

Reading the data
Reading the data
Extracting num_rows from ratings
Preprocessing the dataset
Similar User: 3
Model restored


,attraction_id,Recommendation Score
0,1,0.525922
1,2,0.525061
2,3,0.531307
3,5,0.552330
4,6,0.520033


In [7]:
# %%capture
final = dict()
final['timeofday'] = []
final['image'] = []
final['name'] = []
final['location'] = []
final['avg_price'] = []
final['rating'] = []
final['category'] = []

for i in range(1,(end_date - begin_date).days+2):
    for j in range(2):
        final['timeofday'].append('Morning')
    for j in range(2):
        final['timeofday'].append('Evening')

for i in range(len(final['timeofday'])): 
    if i%4 == 0: 
        final = top_recc(filtered, final)
    else:
        final = find_closest(filtered, final['location'][-1], final['timeofday'][i], final)


Item no.: 1 --> Item name = Nguyen Art Gallery - Vietnam Artworks & Paintings
Evaluating...
Starting Download...

Errors: 0


Item no.: 1 --> Item name = Cafe Goethe
Evaluating...
Starting Download...

Errors: 0



## Final output

In [8]:
days = (end_date - begin_date).days + 1
display(final_output(days,final))

name:
0
['Cháo hồng lương sử', 'Nguyen art gallery - vietnam artworks & paintings', 'Phở gà văn miếu 17', 'Cafe goethe']


name:
1
['Cộng cà phê đường điện biên phủ', 'Bảo tàng lịch sử quân sự việt nam', 'Vườn hoa lênin', 'Bảo tàng mỹ thuật việt nam']


